In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage import filters, measure, morphology
import math
from skimage.measure import label, regionprops_table
from PIL import Image
import colorsys
from skimage import img_as_ubyte, exposure
from os import listdir
from os.path import isfile, join
import os

In [2]:
cytoPath = "/Volumes/Aorta/aorta/segmentations/cellSize/tiles/tilesCellSize/tissueAnalyzer"
savePath = "/Volumes/Aorta/aorta/results/cellSizeStatistics/images"

In [3]:
cytoFolders = [x[0] for x in os.walk(cytoPath)]
cytoFolders = sorted(cytoFolders)

In [10]:
for cytoFolder in cytoFolders:
    os.chdir(cytoFolder)
    dfResults = pd.DataFrame()
    files = [f for f in listdir(cytoFolder) if isfile(join(cytoFolder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    age = cytoFolder.split("/")[-2]
    aorta = cytoFolder.split("/")[-1]
    for file in files:
        masks = np.load(join(cytoFolder, file), allow_pickle=True).item()['masks']
        prop_dict = regionprops_table(masks, properties=['label','area', 'centroid', 'major_axis_length', 'minor_axis_length', "orientation"])
        dfHelp = pd.DataFrame(prop_dict)
        dfHelp['age'] = age
        dfHelp['aorta'] = aorta
        dfHelp['file'] = file
        dfResults = pd.concat([dfResults, dfHelp])

    if len(dfResults.index) != 0:
        dfResults.to_csv(join(cytoFolder, "resultsCellSize_" + age + "_" + aorta + "unfiltered.csv"), index=False)